## Spatial Modeling and Analytics

### Segment 4 of 5
# Spatial Analysis in R

## Reminder
<a href="#/slide-2-0" class="navigate-right" style="background-color:blue;color:white;padding:8px;margin:2px;font-weight:bold;">Continue with the lesson</a>

<br>
</br>
<font size="+1">

By continuing with this lesson you are granting your permission to take part in this research study for the Hour of Cyberinfrastructure: Developing Cyber Literacy for GIScience project. In this study, you will be learning about cyberinfrastructure and related concepts using a web-based platform that will take approximately one hour per lesson. Participation in this study is voluntary.

Participants in this research must be 18 years or older. If you are under the age of 18 then please exit this webpage or navigate to another website such as the Hour of Code at https://hourofcode.com, which is designed for K-12 students.

If you are not interested in participating please exit the browser or navigate to this website: http://www.umn.edu. Your participation is voluntary and you are free to stop the lesson at any time.

For the full description please navigate to this website: <a href="../../gateway-lesson/gateway/gateway-1.ipynb">Gateway Lesson Research Study Permission</a>.

</font>

In [ ]:
# This code cell starts the necessary setup for Hour of CI lesson notebooks.
# First, it enables users to hide and unhide code by producing a 'Toggle raw code' button below.
# Second, it imports the hourofci package, which is necessary for lessons and interactive Jupyter Widgets.
# Third, it helps hide/control other aspects of Jupyter Notebooks to improve the user experience
# This is an initialization cell
# It is not displayed because the Slide Type is 'Skip'

from IPython.display import HTML, IFrame, Javascript, display
from ipywidgets import interactive
import ipywidgets as widgets
from ipywidgets import Layout

import getpass # This library allows us to get the username (User agent string)

# import package for hourofci project
import sys
sys.path.append('../../supplementary') # relative path (may change depending on the location of the lesson notebook)
import hourofci

# load javascript to initialize/hide cells, get user agent string, and hide output indicator
# hide code by introducing a toggle button "Toggle raw code"
HTML(''' 
    <script type="text/javascript" src=\"../../supplementary/js/custom.js\"></script>
    
    <style>
        .output_prompt{opacity:0;}
    </style>
    
    <input id="toggle_code" type="button" value="Toggle raw code">
''')

In [ ]:
#Raster Data

library(geodata)
library(ggplot2)
library(raster)
library(terra)
library("tidyr")

##1.  Wrangle, Display and Plot the Raster
plot(raster.dem)
tmax.raster <- cmip6_world("ACCESS-ESM1-5", "126", "2061-2080", var="tmax", res=5,
                     path=tempdir())

### Plot Raster
plot(tmax.raster)

### Animate All Bands
animate(tmax.raster.subset)

## 2. Display Raster Information
paste0("Raster CRS: ", crs(tmax.raster))
paste0(dim(tmax.raster)[1], ' x ', dim(tmax.raster)[2], 
       ' x ', dim(tmax.raster)[3])
paste0("Raster Resolution: ", res(tmax.raster)[1], ' x ', res(tmax.raster)[2])

### Subset Bands
tmax.raster.b1_3 <- subset(tmax.raster, 1:3)

### Subset by Extent
print(paste0("Current Extent: ", ext(tmax.raster)))
## Define a Bounding Box Over CONUS
bbox <- ext(-124.736342, -66.945392,  24.521208, 49.382808)
tmax.raster.subset <- crop(tmax.raster, bbox)

plot(tmax.raster.subset)


## 3. Extract Bands and Plot Data
tmax.subset.df <- as.data.frame(tmax.raster.b1_3 , xy=FALSE, cells=FALSE, na.rm=NA)
tmax.df.long <- tmax.subset.df %>%
  pivot_longer(colnames(tmax.subset.df)) %>% 
  as.data.frame()
  
ggplot(tmax.df.long, aes(value, fill=name)) + 
  geom_histogram( color="#e9ecef", alpha=0.6, position = 'identity')

## 4. Compute Summary Statistics
tmax.stdev <- stdev(tmax.raster.subset)
tmax.stdev
plot(tmax.stdev, main = "Standard Deviation of Max. Temperature over 10 years")


## 5. Filter Pixels by Value
tmax.stdev.levels <- clamp(tmax.stdev, lower=4, upper=Inf)
plot(tmax.stdev.levels, main = "Standard Deviation of Max. Temperature over 10 years")

tmax.stdev.levels
### Make Plots in A Loop by Filtering
for (i in c(5,6,7,8,9,10,11,12)){
  tmax.stdev.band <- clamp(tmax.stdev, lower=i, upper=Inf, values=FALSE)
  ## Add New STD Level as a New Band
  tmax.stdev.levels <- c(tmax.stdev.levels, tmax.stdev.band)
}
## Note the Differnce in Layers
print(tmax.stdev.levels)
## Animate Levels of Tmax STD
animate(tmax.stdev.levels)


In [ ]:
#Vector Data

## 0. Import Packages
library(ggplot2)
library(leaflet.esri)
library(sf)

## 1. Sudden Infant Death Syndrome (SIDS) sample data for North Carolina counties
sids.nc <- st_read(system.file("shape/nc.shp", package="sf"))

### Print the first 3 rows of the dataset
head(sids.nc, 3)

##  2. Make a static plot of the data
plot(sids.nc)
### Make a static plot for one variable SID79
plot(sids.nc["SID79"], breaks = "jenks")

## 3. Plot New Born Counts Against SIDS as a Line Plot
ggplot(sids.nc, aes(x=NWBIR79, y=SID79))  +
  geom_point( size = 3) + geom_line(size = 1, alpha=0.4, color = 'red') +
  xlab("Number of New Borns") + ylab("Number of SIDS Incidents") +
  theme_light()

## 4. Make A Map with ggplot2
p <- ggplot() +
  geom_sf(data = sids.nc, aes(fill = SID79)) +
  scale_fill_gradientn(colours = hcl.colors(10, alpha = 0.5))
p

## 5. Defining Subsets By Indexing
### Make Subsets by Row Indices
sids.nc.row.subset <- sids.nc[1:15,]
p2 <- ggplot() +
  geom_sf(data = sids.nc ) + 
  geom_sf(data = sids.nc.row.subset, aes(fill = SID79) )
p2

## 6. Defining Subsets by Intersection with Bounding Box
bbox <- st_bbox(sids.nc)
print(bbox)

### Define a New BOunding Box to Intersect
new_bbox <- st_bbox(c(xmin = -80,
              ymin = 35,
              xmax = -77, 
              ymax = 36), crs = st_crs(sids.nc))
### Convert the Bounding Box to A Feature
new_bbox <- st_as_sfc(new_bbox)

### Intersect Features
sids.subset.bbox <- st_intersection(sids.nc, new_bbox)

p2 <- ggplot() +
  geom_sf(data = sids.nc ) + 
  geom_sf(data = sids.subset.bbox, aes(fill = SID79) )
p2


## 7. Reproject Data to WGS84 and Make Interactive Map
sids.nc.wgs84 <- st_transform(sids.nc,  crs = 4326)

### Make a Leaflet map of the data
L <- leaflet(sids.nc.wgs84) %>%
  addEsriBasemapLayer(esriBasemapLayers$Imagery) %>%
  addPolygons(color = "#000000", weight = 1, opacity = 1.0, fillOpacity = 0.5,
              fillColor = ~colorQuantile("YlOrRd", SID79)(SID79),
              highlightOptions = highlightOptions(color = "white",
                                                  bringToFront = TRUE),
              popup = paste0("<b>SID Cases in 1979:</b> ", sids.nc.wgs84$SID79,
                             "<br>", "<b>New Births in 1979</b>: ", 
                             sids.nc.wgs84$NWBIR79))

L




<font size="+1"><a style="background-color:blue;color:white;padding:12px;margin:10px;font-weight:bold;" 
href="sma-6.ipynb">Click here to go to the next notebook.</a></font>